In [4]:
import glimpse
from glimpse.imports import datetime,np,os
import glob
from PIL import Image
from PIL.ExifTags import TAGS
import numpy as np

First, we will define the camera model. We can get the intrinsic parameters using image meta-data, and the extrinsic parameters from surveyed gps data for the camera's location and an initial guess at the camera's view orientation that we can refine later.

In [5]:
exifinfo = {}
img_path = '/home/dunbar/Research/wolverine/data/cam_cliff/images/cam_cliff_0497.JPG'
image = Image.open(img_path)
exif_info = image._getexif()
for (tag,value) in exif_info.items():
    tagname = TAGS.get(tag,tag)
    exifinfo[tagname] = value

For the purposes of GLIMPSE, we will need the following from the dictionary defined above: Image Dimensions (pixels), Focal Length (mm), Sensor Dimensions (mm), and Date (datetime format).
We can typically get all but the Sensor Dimensions from the images themselves. In this case I had to look it up.

In [6]:
image_focal = (exifinfo['FocalLength'][0]/exifinfo['FocalLength'][1])
imagewidth = int(exifinfo['ExifImageWidth'])
imageheight = int(exifinfo['ExifImageHeight'])
image_date = exifinfo['DateTime']
sensorx,sensory = 35.9,24

In [7]:
cam_coords = [393506.713,6695855.64 ,961.3370] #Easting, Northing, Elevation
cam_viewdir = [72.08866164934302, -24.53,0.35382] # An intial guess at the camera's viewing pose in yaw, pitch, roll

In [8]:
cliff_camera = glimpse.Camera(vector=None,xyz=cam_coords,viewdir=cam_viewdir,imgsz=(imagewidth,imageheight),fmm=(image_focal,image_focal),sensorsz=(sensorx,sensory))

Now that we have our camera model defined, we can "fine tune" it's parameters using Glimpse's optimize module. For this, we need real world coordinate to image coordinate coorespondances in respective arrays.

In [10]:
cliff_imgpts = np.array([[5193, 549],[3101, 642],[3680, 2456],[6153.0, 2297.0]])
cliff_worldpts = np.array([[408245.86,6695847.03,1560 ],[416067.22,6707259.97,988],[393506.713, 6695855.641, 961.337],[394569.509, 6695550.678, 621.075]])

cliff_points = glimpse.optimize.Points(cliff_camera,cliff_imgpts,cliff_worldpts)
cliff_cam_params = glimpse.optimize.Cameras(cliff_camera,cliff_points)
cliff_cam_params = cliff_cam_params.fit()

5738.996927352331

ValueError: zero-size array to reduction operation maximum which has no identity

Now that there is  a reasonably defined camera model, we can create an observer object which holds the images on which GLIMPSE will perform feature tracking. 

In [11]:
img_path = '/home/dunbar/Research/wolverine/data/cam_cliff/images'
images = glob.glob(os.path.join(img_path,'*.JPG'),recursive=True)
images = [glimpse.Image(path=imagepath,cam=cliff_camera.copy()) for imagepath in images]
images.sort(key= lambda img: img.datetime ) # sort by datetime


In [12]:
cliffobserver = glimpse.Observer(images,cache=False)

With our observer in good shape, we can now proceed to prepare the DEM

In [13]:
dem_path = '/home/dunbar/Research/wolverine/data/dem'
dem = glob.glob(os.path.join(dem_path,'*.tif'))
dem = glimpse.Raster.read(dem[0])
dem.crop(zlim=(0,np.inf))
dem.fill_crevasses(mask=~np.isnan(dem.Z), fill=True)
dem.fill_circle(cliffobserver.xyz,radius=100)

In [ ]:
viewshed = dem.copy()
viewshed.Z = np.ones(dem.shape, dtype=bool)
viewshed.Z &= dem.viewshed(cliffobserver.xyz)

[warning] DEM cells not square (1.9272510849894067, 1.9272510849894346) - may lead to unexpected results
